In [1]:
# only permit vertex cover here

In [2]:
import os
os.listdir('/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs')

['legacy',
 'Testing',
 'LiftMP_runs',
 '230924_hparam2',
 '230927_snapshot',
 '230924_hparam',
 '230924_hparam_TU_multiarch',
 '230924_hparam_TU',
 '230926_finetune_ER_runs']

In [3]:
run_folders = ['LiftMP_runs', '230927_snapshot/230925_TUsmall_GAT_VC', '230927_snapshot/230925_TUsmall_GIN_VC', 
               '230927_snapshot/230925_generated_preset_cut', '230927_snapshot/230925_TUlarge_all_cut', '230927_snapshot/230925_TUsmall_GCNN_VC',
                 '230927_snapshot/230925_generated_liftMP_VC', '230927_snapshot/230925_TUlarge_all_VC', '230927_snapshot/230925_generated_preset_VC', 
                 '230927_snapshot/230925_TUsmall_liftMP_VC', '230927_snapshot/230925_TUsmall_GatedGCNN_VC', '230927_snapshot/230925_generated_liftMP_cut', '230926_finetune_ER_runs']


In [4]:
import json
import pandas as pd
import numpy as np

folder_path = '/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs'
model_list = [os.path.join(folder_path, run_folder, x) for run_folder in run_folders for x in os.listdir(os.path.join(folder_path, run_folder))  ]


rows = []

errored = []
for model_folder in model_list:
    try:
        with open(os.path.join(model_folder, 'params.txt'), 'r') as f:
            model_args = json.load(f)
        if model_args['problem_type'] != 'vertex_cover':
            continue
        
        losses = np.load(os.path.join(model_folder, 'valid_scores.npy'))
        test_losses = np.load(os.path.join(model_folder, 'test_scores.npy'))
        modeldict = model_args #{x: model_args[x] for x in params}
        modeldict['max_valid_score'] = max(losses)
        modeldict['max_valid_epoch'] = np.argmax(losses)
        modeldict['scores'] = test_losses[np.argmax(losses)]
        modeldict['baseline'] = False

        rows.append(modeldict)
        
    except Exception as e:
        print(f'{e} is wrong w/ {model_folder}')
        errored.append(model_folder)

[Errno 2] No such file or directory: '/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:d7295454a183410793f153014e74c48fb81051635748e2fa4358e5e62ff747c8/valid_scores.npy' is wrong w/ /home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:d7295454a183410793f153014e74c48fb81051635748e2fa4358e5e62ff747c8
[Errno 2] No such file or directory: '/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:959f4a30685c103490271cfaa513d24bffe58e4ff7f56301797ac56a2ab76978/valid_scores.npy' is wrong w/ /home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:959f4a30685c103490271cfaa513d24bffe58e4ff7f56301797ac56a2ab76978
[Errno 2] No such file or directory: '/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:af7aa021f2876f5b35b4888f53a3bc63a8846a1ae09ca8bd2d6f8c1ee2f9923a/valid_scores.npy' is wrong w/ /home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:af7aa021f2876f5b35b4888f53a

In [5]:
len(errored)

162

# load in baselines
baselines = pd.read_csv('vc_baseline_scores.csv')

# unwind them 
for i, baseline in baselines.iterrows():
    #print(baseline.index)
    for col in baselines.keys():
        if col == 'Unnamed: 0':
            continue
        ds = col
        gen_n = np.nan
        if len(col.split('@@')) > 1:
            ds, gen_n = col.split('@@')
            gen_n = int(gen_n)

        row = {'dataset': ds, 'gen_n': gen_n, 'model_type': baseline['Unnamed: 0'], 'scores': baseline[col], 'baseline': True}
        #print(row)
        rows.append(row)


In [6]:
df = pd.DataFrame.from_records(rows)

In [7]:
df

,problem_type,seed,prefix,model_type,num_layers,repeat_lift_layers,num_layers_project,rank,dropout,hidden_channels,...,RB_k,positional_encoding,pe_dimension,split_seed,log_dir,device,max_valid_score,max_valid_epoch,scores,baseline
0,vertex_cover,0,230925_generated_liftMP_VC,LiftMP,8,None,4,16,0.0,32,...,"[5, 21]",random_walk,8,0,training_runs/230925_generated_liftMP_VC/param...,cuda,-55.767000,4,-55.710000,False
1,vertex_cover,0,230925_TUsmall_liftMP_VC,LiftMP,16,None,4,4,0.0,32,...,"[5, 21]",random_walk,2,0,training_runs/230925_TUsmall_liftMP_VC/paramha...,cuda,-15.440000,98,-16.810000,False
2,vertex_cover,0,230925_TUsmall_liftMP_VC,LiftMP,16,None,4,16,0.0,32,...,"[5, 21]",random_walk,8,0,training_runs/230925_TUsmall_liftMP_VC/paramha...,cuda,-18.916667,14,-20.000000,False
3,vertex_cover,0,230925_generated_liftMP_VC,LiftMP,8,None,4,32,0.0,32,...,"[5, 21]",random_walk,16,0,training_runs/230925_generated_liftMP_VC/param...,cuda,-125.951000,1,-126.140000,False
4,vertex_cover,0,230925_TUsmall_liftMP_VC,LiftMP,16,None,4,4,0.0,32,...,"[5, 21]",laplacian_eigenvector,2,0,training_runs/230925_TUsmall_liftMP_VC/paramha...,cuda,-8.736842,8,-7.789474,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,vertex_cover,0,230926_finetune_ER_runs,LiftMP,8,None,4,8,0.0,32,...,"[5, 21]",random_walk,4,0,training_runs/230926_finetune_ER_runs/paramhas...,cuda,-8.736842,1,-7.736842,False
761,vertex_cover,0,230926_finetune_ER_runs,LiftMP,16,None,4,8,0.0,32,...,"[5, 21]",random_walk,4,0,training_runs/230926_finetune_ER_runs/paramhas...,cuda,-26.000000,19,-27.321429,False
762,vertex_cover,0,230926_finetune_ER_runs,LiftMP,16,None,4,8,0.0,32,...,"[5, 21]",random_walk,4,0,training_runs/230926_finetune_ER_runs/paramhas...,cuda,-15.420000,0,-16.760000,False
763,vertex_cover,0,230926_finetune_ER_runs,LiftMP,16,None,4,8,0.0,32,...,"[5, 21]",random_walk,4,0,training_runs/230926_finetune_ER_runs/paramhas...,cuda,-18.916667,45,-20.016667,False


In [8]:
df['gen_n_max'] = df.gen_n.apply(lambda x: x[-1] if isinstance(x,list) else x)
df.gen_n = df.gen_n.apply(lambda x: x[0] if isinstance(x,list) else x)


In [9]:
df.keys()
# 'problem_type', 'seed',  'prefix', 'RB_n', 'RB_k', 'log_dir',
relevant_keys = [ 'model_type', 'num_layers',
       'repeat_lift_layers', 'num_layers_project', 'rank', 'vc_penalty', 'gen_n',
       'dataset', 'infinite',  'positional_encoding', 'pe_dimension',
       'max_valid_score', 'max_valid_epoch',
       'scores', 'finetune_from']

In [15]:
df = df[df.model_type== 'LiftMP']

In [25]:
dataset_by_arc = pd.DataFrame()
dss = ['BarabasiAlbert', 'ErdosRenyi', 'PowerlawCluster', 'WattsStrogatz']


for (mt, ds, gen_n), group in df[df.finetune_from.isna() & ~df.dataset.isin(dss)].groupby(['model_type', 'dataset', 'gen_n']):
    dataset_by_arc.at[f'{ds}', 'LiftMP Base'] = group['scores'].max()
    dataset_by_arc.at[f'{ds}', 'LiftMP Base best epoch'] = group.max_valid_epoch[group['scores'].idxmax()]

dataset_by_arc[[k for k in dataset_by_arc.keys() if k != 'SDP lift']]#.style.highlight_max(color = 'green', axis = 1)

,LiftMP Base,LiftMP Base best epoch
COLLAB,-67.504000,35.0
ENZYMES,-20.000000,14.0
IMDB-BINARY,-16.760000,52.0
MUTAG,-7.736842,0.0
PROTEINS,-25.294643,48.0


In [26]:
for (mt, ds, gen_n), group in df[~df.finetune_from.isna() & ~df.dataset.isin(dss)].groupby(['model_type', 'dataset', 'gen_n']):
    dataset_by_arc.at[f'{ds}', 'Finetuned score'] = group['scores'].max()
    dataset_by_arc.at[f'{ds}', 'Finetuned best epoch'] = group.max_valid_epoch[group['scores'].idxmax()]


dataset_by_arc[[k for k in dataset_by_arc.keys() if k != 'SDP lift']]#.style.highlight_max(color = 'green', axis = 1)

,LiftMP Base,LiftMP Base best epoch,Finetuned score,Finetuned best epoch
COLLAB,-67.504000,35.0,-67.546000,29.0
ENZYMES,-20.000000,14.0,-20.016667,49.0
IMDB-BINARY,-16.760000,52.0,-16.760000,0.0
MUTAG,-7.736842,0.0,-7.736842,0.0
PROTEINS,-25.294643,48.0,-25.401786,0.0
